In [95]:
#With this cell, all validation/exception handling functions are implemented
#

def check_pos_int(x, case):
    """Since several functions in this implementation of the subnumpy library take positive integers as input, it makes sense
    to implement the exception handling once in a function to avoid redundancies in the code and make it more modular.
    This function is used for checking the input numbers within tuples. It distinguished between case 1, 2, and 3 to give a more precise
    and descriptive exception string.


    Input:
    Given input x by the user for a function, where a positive integer is expected.
    Case distinguished between check of simple numeric input (1), first element of tuple (2), and second element of tuple (3).
    """
    m_exception = ""
    if (not isinstance(case, int)) or case > 3 or case < 1:
        raise Exception("No such case " + f"{case} " + "second argument must be integer between 1 and 3 inclusively.")
    elif case == 1:
        m_exception = "The argument given ("
    elif case == 2:
        m_exception = "The first element of the tuple ("
    elif case == 3:
        m_exception = "The second element of the tuple ("

    if not isinstance(x, int):
        raise Exception(m_exception + f"{x}" + ") is no integer.")
    elif isinstance(x, int) and x < 1:
        raise Exception(m_exception + f"{x}" + ") is an integer smaller than 1.")

def check_tuple(t):
    """
    This function checks the validity of numeric tuples. The input shall be a tuple with two elements and consist of positiv integers.
    For the letter, the fuction check_pos_int is used on both elements of the argument - in case the former is fulfilled.
    """
    if not isinstance(t, tuple):
        raise Exception("The second argument(" + f"{t}" + ")is not a tuple.")
    elif len(t) != 2:
        raise Exception("The second argument(" + f"{t}" + ")is not a tuple of length 2.")
    t0 = t[0]
    t1 = t[1]
    check_pos_int(t0, 2)
    check_pos_int(t1, 3)

def check_arr(v):
    """
    This vector iterates through the vector and checks, if it has only numeric values. Raises exception if requirement not fulfilled.
    """
    dim = len(v)
    for i in range(dim):
        if not(isinstance(v[i], int) or isinstance(v[i], float)):
            raise Exception("The vector ("+ f"{v}" +") does not consist of numeric values only.")

def check_matrix_vector(v_m):
    """
    A matrix/vector is implemented as an array of array, in which each array is one row of the matrix/vector.
    To be a valid matrix/vector, m must be an array consisting of only numeric arrays of which all have the same length.
    """
    n_rows = len(v_m)
    if not isinstance(v_m, list):
        raise Exception("Wrong input type - shall be array")
    if n_rows < 1:
        raise Exception("Matrix has no values.")
    for i in range(n_rows-1):
        n1 = len(v_m[i])
        n2 = len(v_m[i+1])
        if n1 != n2:
            raise Exception("Matrix/Vector is not well defined - shall have same number of row entries for every row.")
    for row in range(n_rows-1):
        for col in range(n1):
            if not(isinstance(v_m[row][col], int) or isinstance(v_m[row][col], float)):
                raise Exception("The vector ("+ f"{v}" +") does not consist of numeric values only.")

def augmented(matrix, vector):
    """
    This function concatenates a matrix and a vector to an augmented matrix. Validity checks are made before this function is called.
    """
    ####error handling tbd
    m = len(matrix)
    for i in range(m):
        matrix[i].append(vector[i])
    return matrix

def swap(matrix, index):
    """
    This function helps to swap rows in respect to the pivot values so that zeros are swapped to the bottom rows.
    It is called for bring rows in the right order regarding the values in the column given as the input argument <<index>>.

    """
    m, n = snp.shape(matrix)
    #the check for leading zeros in a certain column must only be done for the rows down from the index_th row because the row above can be assumed to be in the echelon form already
    for row_idx in range(index, m):
        #the swap must only be carried out if the respective value is equal to zero
        if matrix[row_idx][index] != 0:
            continue
        else:
            #for the swap, a row with a value in the index_th position not equal to zero is searched
            for row_comp in range(row_idx+1, m):
                if matrix[row_comp][index] != 0:
                    #for a potential swap, one row must be saved in another variable
                    row_store = matrix[row_idx]
                    #swap is carried out by exchanging the position of the rows
                    matrix[row_idx] = matrix[row_comp]
                    matrix[row_comp] = row_store
                    break
    return matrix

#TBD: Echelon
def echelon(matrix):
    """
    This function calculates the echelon form of a square matrix.
    """
    check_matrix_vector(matrix)
    m, n = snp.shape(matrix)
    #the coefficient matrix has to be square: m must be equal to n+1; note that the input of this function is the augmented matrix [That is a requirement np.lin_alg also poses.]
    if m+1 != n:
        raise Exception("Format of matrix is wrong - it must be square.")
    #there is no proper echelon form form if the matrix given as an input is either a row or a column vector
    if m<2 or n<2:
        raise Exception("Matrix that was given as input is too small - both dimesions must be greater than 1.")
    i = 0
    while i < m-1 and i < n:
        matrix = swap(matrix, i)
        #because swap was called before: if ???"anchor"/pivot??? value is zero, echelon is done for that column
        if matrix[i][i] != 0:
            #for "anchor" value, all values below must be set to zero to create the echelon form
            for iter_n in range(i, m-1):
                #if any value under the anchor value is zero, the ones below are as well because swap was called before - next anchor/row can be tackled
                if matrix[iter_n+1][i] == 0:
                    break
                #row operation: row_x = row_x - ( factor * row_(x-1) )
                factor = matrix[iter_n+1][i] / matrix[i][i]
                matrix[iter_n+1] = [a - (factor * b) for a,b in zip(matrix[iter_n+1], matrix[i])]
        i = i + 1
    #to cope with rounding errors that result from float operations, values are rounded to 10 decimal places
    for row in range(m):
        new_row_pre = matrix[row]
        matrix[row] = [round(v, 10) for v in new_row_pre]
    return matrix

def reduced_echelon(matrix):
    """
    After the echelon form of the matrix is calculted, this function calculates the reduced echelon form.
    """
    #first, matrix is transformed into the echelon form, which is the foundation for the transformation into rref
    matrix = echelon(matrix)
    m, n = snp.shape(matrix)
    #iterating from the lower right to the upper left corner
    i = m - 1
    while i >= 0:
        #if that value is 0, there cannot be a reduction in that column for the rows above based on it.
        if matrix[i][i] != 0:
            for iter_n in range(i, 0, -1):
                if matrix[i][i] == 0:
                    break
                factor = matrix[iter_n-1][i] / matrix[i][i]
                matrix[iter_n - 1] = [a - (factor * b) for a, b in zip(matrix[iter_n - 1], matrix[i])]
        i = i - 1

    #finalizing to rref by deviding row values so that coefficients matrix is diagonal with ones
    for n in range(m):
        if matrix[n][n] != 0:
            value = 1 / ( matrix[n][n] )
            matrix[n] = [value * a for a in matrix[n]]

    #Rounding to cope with floating-point precision errors
    for row in range(m):
        new_row_pre = matrix[row]
        matrix[row] = [round(v, 5) for v in new_row_pre]
    return matrix

def rref_check(matrix):
    condition = 1
    m,n = snp.shape(matrix)
    for r in range(m):
        for i in range(m):
            if (r != i and matrix[r][i] != 0):
                condition = 0
    if condition == 0:
        return False
    return True

def solution_case(matrix):
    """
    This function determines based on the input - the matrix in rref - how many solutions exist.
    Output is the number of the case which is then further processed in the
    """
    
    #@nick, here we also need to adjust the case in which the matrix returned by the reduced_echlon has infinitely many solutions
    
    m, n = snp.shape(matrix)
    #tbd: is in rref?
    solution_case = 0 #1: no solution; 2:infinite number of solution, 3: one solution;

    #no solution, case 1 - occurs when coefficient matrix has zero row but corresponding ??equation value?? is non zero
    for row in matrix:
        for i in range(n-1):
            if (all(value == 0 for value in row[:(n-1)]) and (row[n-1] != 0)):
                solution_case = 1
    #one solution, case 3 - occurs, when matrix in rref and no free variables
    #for efficiency reasons, here, it is only checked, if matrix in rref, case 3, in that case, is only overturned, if there are free variables
    if rref_check(matrix):
        solution_case = 3
    #infinite number of solutions, case 2 - matrix in rref but it has one or more free variables
    for i in range(m):
        if matrix[i][i] == 0:
            solution_case = 2
            break
    if solution_case == 0:
        raise Exception("Reduced echelon form was not possible to be calculated - linear eqaution system is inconsistent")
    return solution_case

def one_solution_solver(matrix):
    """
    This function gets a matrix in rref, for which there is only one solution, as input.
    It gives back the corresponding solution.
    """
    m, n = snp.shape(matrix)
    solution = []
    for i in range(m):
        res = matrix[i][n-1] / matrix[i][i]
        print(res)
        solution.append(res)
    return solution

def infinite_solution_finder(matrix):
    """
    This function gets a matrix in rref with at least one free variable. It than returns one of the possible solutions.
    """
    pass
    #@nick, this is mainly what you need to implement




In [100]:
class Subnumpy(object):

    #The follwing function is
    def ones(self, x):
        """
        This function creates an array of length x containing only ones.
        Input: x, a positive integer
        Output: array of length x containing only ones
        """
        check_pos_int(x, 1)
        out = []
        out = [1 for i in range(x)]
        return out

    def zeros(self, x):
        """
        This function creates an array of length x containing only zeros.
        Input: x, a positive integer
        Output: array of length x containing only zeros
        """
        check_pos_int(x, 1)
        out = []
        out = [0 for i in range(x)]
        return out

    def reshape(self, arr, t):
        """
        #TBD
        """
        check_arr(arr)
        check_tuple(t)
        m = t[0]
        n = t[1]
        matrix = []
        if m*n != len(arr):
            raise Exception("Length of vector and matrix dimension do not match.")
        else:
            for m_rows in range(m):
                x1 = m_rows * n
                x2 = (m_rows + 1) * n
                matrix.append(arr[x1:x2])
            return matrix

    def shape(self, matrix):
        """
        This function returns the dimensions m and n of a mxn matrix.
        Before calculating, it has to be checked if the matrix-requirements for the input sre fulfilled.
        Input: Valid matrix.
        Output: Dimensions m and n of mxn matrix
        """
        check_matrix_vector(matrix)
        m = len(matrix)
        n = len(matrix[0])
        return (m, n)

    def append(self, matrix1, matrix2):
        pass

    def get(self, v_m, dim):
        """
        This function returns the values pecified by the coordinate point (row,column) of the array provided
        (can be vector or matrix).
        Input: Valid Vector or Matrix and a valid tuple specifiying the coordinate of the value of interest.
        Note that the input must be given in natural numbers starting with (1, 1) for the top left entry.
        Output: Numeric value
        """
        check_matrix_vector(v_m)
        m, n = snp.shape(v_m)
        check_tuple(dim)
        if (dim[0]) > m or (dim[1]) > n:
            raise Exception("Dimension input does not exists for the given matrix/vector.")
        else:
            dim1_pre, dim2_pre = dim
            dim1 = dim1_pre - 1
            dim2 = dim2_pre - 1
            value = v_m[dim1][dim2]
        return value

    def add(self, v_m1, v_m2):
        check_matrix_vector(v_m1)
        check_matrix_vector(v_m2)
        m1, n1 = snp.shape(v_m1)
        m2, n2 = snp.shape(v_m2)
        if m1 != m2 and n1 != n2:
            raise Exception("Operation not possible - both matrices/vectors given do not have the same dimesnsions.")
        result = []
        for i in range(0,m1):
            result.append([a + b for a, b in zip(v_m1[i], v_m2[i])])
        return result

    def subtract(self, v_m1, v_m2):
        check_matrix_vector(v_m1)
        check_matrix_vector(v_m2)
        m1, n1 = snp.shape(v_m1)
        m2, n2 = snp.shape(v_m2)
        if m1 != m2 and n1 != n2:
            raise Exception("Operation not possible - both matrices/vectors given do not have the same dimesnsions.")
        result = []
        for i in range(0,m1):
            result.append([a - b for a, b in zip(v_m1[i], v_m2[i])])
        return result

    def dotproduct(self, v_m1, v_m2):
        check_matrix_vector(v_m1)
        check_matrix_vector(v_m2)
        m1, n1 = snp.shape(v_m1)
        m2, n2 = snp.shape(v_m2)
        if n1 != m2:
            raise Exception("The combination of the input vectors/matrices do not fulfill the requirements for the dotprouct.\n The bumber of rows in the first input must be equal to the number of columns of the second input.")
        #initialize resulting matrix/vector
        res_dim = m1, n2
        res = []
        zero_row = snp.zeros(n2)
        for i in range(m1):
            res.append(zero_row)
        for m in range(m1):
            for n in range(n2):
                sum = 0
                for iterate in range(n1):
                    sum = sum + ( v_m1[m][iterate] * v_m2[iterate][n] )
                res[m][n] = sum
        return res

    def linalg_solver(self, matrix, vector):
        #check_matrix_vector(matrix)
        #check_matrix_vector(vector)
        if len(matrix) != len(vector):
            raise Exception("The dimension of the input - matrix & vector - do not fit.")
        aug_matrix = augmented(matrix, vector)
        rref = reduced_echelon(aug_matrix)
        print(rref)
        res_case = solution_case(rref)
        if res_case == 1:
            print("There is no solution to the linear equation system.")
            return
        if res_case == 2:
            pass
        if res_case == 3:
            return one_solution_solver(matrix)
        #tbd: solution finder


    def show_matrix(v_m):
        """
        This function is an addition to the tasks given on the assignment sheet. It can be used to display matrices or vectors in a more comprehensive and illustrated way.
        """
        check_matrix_vector(v_m)
        #error handling
        print(v_m)


In [101]:
snp = Subnumpy()
x = snp.zeros(10)

In [102]:

# Consistent System with Unique Solution
matrix_unique_solution = [[2, -1, 3, 4],
                            [1, 2, -1, 5],
                           [3, 1, 2, 7]]
rref_unique_solution = [[1, 0, 0, 1],
                        [0, 1, 0, 2],
                        [0, 0, 1, 3]]

# Consistent System with Infinite Solutions
matrix_infinite_solutions = [
                            [1, 2, 3, 4],
                            [2, 4, 6, 8],
                            [3, 6, 9, 12]]
rref_infinite_solutions = [[1, 2, 3, 4],
                           [0, 0, 0, 0],
                           [0, 0, 0, 0]]

# Inconsistent System
matrix_inconsistent = [[1, 2, 3, 4],
                       [2, 4, 6, 9],
                       [3, 6, 9, 12]]
rref_inconsistent = [[1, 2, 3, 4],
                     [0, 0, 0, 1],
                     [0, 0, 0, 0]]

# Overdetermined System
matrix_overdetermined = [[1, 2, 3, 4],
                         [2, 3, 4, 5],
                         [3, 4, 5, 6]]
rref_overdetermined = [[1, 0, -1, 2],
                       [0, 1, 2, -1],
                       [0, 0, 0, 0]]

# Test your reduced_echelon function
#ALL OF THE FOLLOWING CHECKED WITH REAL RESULTS -> IS RIGHT
result_unique_solution = reduced_echelon(matrix_unique_solution)
print(result_unique_solution)


result_infinite_solutions = reduced_echelon(matrix_infinite_solutions)
print(result_infinite_solutions)

result_inconsistent = reduced_echelon(matrix_inconsistent)
print(result_inconsistent)

result_overdetermined = reduced_echelon(matrix_overdetermined)
print(result_overdetermined)

# Check if the results match the expected RREFs
#assert result_unique_solution == rref_unique_solution
#assert result_infinite_solutions == rref_infinite_solutions
#assert result_inconsistent == rref_inconsistent
#assert result_overdetermined == rref_overdetermined

#print("All tests passed!")


[[1.0, 0.0, 1.0, 2.6], [0.0, 1.0, -1.0, 1.2], [0.0, 0.0, 0.0, -2.0]]
[[1.0, 2.0, 3.0, 4.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]]
[[1.0, 2.0, 3.0, 4.0], [0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0]]
[[1.0, 0.0, -1.0, -2.0], [-0.0, 1.0, 2.0, 3.0], [0.0, 0.0, 0.0, 0.0]]


In [132]:
import numpy as np


a = [[1, 0, 0], [0, 0, 0], [0, 0, 8]]
b = [9,0,34]

nnp_solution = np.linalg.solve(np.array(a), np.array(b))
snp_solution = snp.linalg_solver(a, b)

print(nnp_solution)
print((snp_solution))



LinAlgError: Singular matrix

# New Section

# New Section

In [36]:
snp = Subnumpy()
mrtx = [[0,1,7,1,11],[2,2,1,2,10], [3,4,0,0,19], [1,2,3,1,8]]
mrtx = reduced_echelon(mrtx)
print(mrtx)

mrtx_2 = [[1,-2,3,9],[-1,3,0,-4],[2,-5,5,17]]
mrtx_2 = reduced_echelon(mrtx_2)
print(mrtx_2)

mrtx_3 = [[1,1,1,2],[0,1,-3,1],[2,1,5,0]]
mrtx_3 = reduced_echelon(mrtx_3)
print(mrtx_3)

mrtx_4 = [[1,1,1,2],[0,1,-3,1],[0,0,0,0]]
mrtx_4 = reduced_echelon(mrtx_4)
print(mrtx_4)



#ech = echelon(mrtx)
#mrtx = swap(mrtx, 0)
#mrtx

#snp.show_matrix(mrtx) 

[[1.0, 0.0, 0.0, 0.0, 11.1052631579], [0.0, 1.0, 0.0, 0.0, -3.5789473684], [0.0, 0.0, 1.0, 0.0, 2.6315789474], [-0.0, -0.0, -0.0, 1.0, -3.8421052632]]
[[1.0, 0.0, 0.0, 1.0], [0.0, 1.0, 0.0, -1.0], [0.0, 0.0, 1.0, 2.0]]
[[1.0, 0.0, 4.0, -2.0], [-0.0, 1.0, -3.0, 4.0], [0.0, 0.0, 0.0, -3.0]]
[[1.0, 0.0, 4.0, 1.0], [0.0, 1.0, -3.0, 1.0], [0, 0, 0, 0]]


In [ ]:

# Example usage:
echelon_matrix_no_solution = [
    [2, 3, 5, 8],
    [0, 1, 2, 3],
    [0, 0, 0, 7]  # No solution
]

echelon_matrix_one_solution = [
    [2, 3, 5, 8],
    [0, 1, 2, 3],
    [0, 0, 3, 4]
]

echelon_matrix_infinite_solutions = [
    [2, 3, 5, 8],
    [0, 1, 2, 3],
    [0, 0, 0, 0]  # Infinite solutions
]

result_no_solution = back_substitution(echelon_matrix_no_solution)
result_one_solution = back_substitution(echelon_matrix_one_solution)
result_infinite_solutions = back_substitution(echelon_matrix_infinite_solutions)

print("Result (No solution):", result_no_solution)
print("Result (One solution):", result_one_solution)
print("Result (Infinite solutions):", result_infinite_solutions)

In [ ]:
x = [1,2,3,4,5,6,7,8,9]
matrix = snp.reshape(x, (3,3))
res = snp.echelon(matrix)
print(res)

In [ ]:
x = [1,0,1,0,1,0,0,0,0]
matrix = snp.reshape(x, (3,3))
res = snp.echelon(matrix)
print(res)

In [ ]:
# Coefficient matrix
a = [[0,4,9,1],[7,1,3,1],[1,5,8,1]]

ech = snp.echelon(a)

# Right-hand side vector
b = np.array([1,2,3])

# Solve the system of linear equations
#x = np.linalg.solve(a, b)

print(x)
ech

In [ ]:
matrix = [[1,2,3], [1,2,3],[1,2,3]]
vect = [4,4,4]
augment = augmented(matrix, vect)
augment

In [ ]:
from sympy import symbols, Eq, solve

# Define the variables
b, c = symbols('b c')

# Define the equation
equation = Eq((-3) * b + (-6) * c, 1)

# Solve the equation
solutions = solve(equation, (b, c))

# Print the solutions
print("Solutions:", solutions)

In [ ]:
for i in range(1,10):
    print(i)

In [ ]:
x = [0,1,2,3,4]
print(x[:4])

In [ ]:
import numpy as np

# Coefficient matrix
a = np.array([[1,2,3],[1,2,6],[7,8,9]])

# Right-hand side vector
b = np.array([1,1,0])

# Solve the system of linear equations
x = np.linalg.solve(a, b)

print(x)

In [ ]:
matrix = [[1,2,3,1],[1,2,3,4],[1,2,1,2]]
n = len(matrix[0])
print((matrix[0][:(n-1)]))
print((matrix[0][n-1]))

In [ ]:
matrix = [[2, -1, 3, 4], [0, 2.5, -2.5, 3], [0, 0, 0, -2]]
i = 2
while i >= 0:
      #if that value is 0, there cannot be a reduction in that column for the rows above based on it.
      if matrix[i][i] != 0:
          for iter_n in range(i, 0, -1):
              if matrix[i][i] == 0:
                  break
              factor = matrix[iter_n-1][i] / matrix[i][i]
              matrix[iter_n - 1] = [a - (factor * b) for a, b in zip(matrix[iter_n - 1], matrix[i])]
      i = i - 1

matrix

In [21]:
matrix = [[1,2,7],[1,2,6],[1,2,4]]
for row in matrix:
    for i in range(3):
        print(row[:i])

[]
[1]
[1, 2]
[]
[1]
[1, 2]
[]
[1]
[1, 2]
